In [2]:
'''
Chat G api version

'''

'\nChat G api version\n\n'

In [3]:
import os
from glob import glob

In [4]:
root_path = "/disk3/eric/VG_arma3_roboflow"
root_path_img = root_path + "/images/*.jpg"

In [5]:
img_paths = sorted(glob(root_path_img))

In [6]:
import base64
import requests

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


#----------------------------------------------
def caption_with_chatg(txt_prompt, image_path,api_key):

    # Getting the base64 string
    base64_image = encode_image(image_path)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4o",
        "messages": [
        {
            "role": "user",
            "content": [
            {
                "type": "text",
                "text": f"{txt_prompt}"
            },
            {
                "type": "image_url",
                "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
                }
            }
            ]
        }
        ],
        "max_tokens": 300
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload).json()
    #
    z = response['choices'][0]['message']['content']
    print("content :",z)
    return z 

In [9]:
import collections
tmp_ = collections.defaultdict()

# option 1 : Describe the image implicitly refer to the ontology below. => 암묵적 허용을 해줘서 서술. 
# option 2 : Describe the image based on the ontology below. => 명시적으로 온톨로지들을 기준으로 정리. 


text_prompt = '''

Describe the image based on the ontology below.

entity: soldier, checkpoint, vehicle, gun, tank


'''



#--- single try
from dotenv import load_dotenv
# Load environment variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

image_path = "/disk3/eric/VG_arma3_roboflow/images/frame_2900.jpg"

caption_r = caption_with_chatg(text_prompt, image_path, api_key)


content : The image depicts two soldiers running along a road in a wooded area. They are holding guns and appear to be in a tactical situation. There is no visible checkpoint, vehicle, or tank in the image.


In [10]:
#--- multiple images

for path in img_paths:
    try:
        print(path)
        img_id = path.split("/")[-1]
        tmp_[img_id] = caption_with_chatg(text_prompt, path, api_key)
        
    except Exception as e:
        print(f"error : {e}")
        pass


/disk3/eric/VG_arma3_roboflow/images/frame_2390.jpg
content : The image depicts an outdoor scene with a dirt road surrounded by trees and vegetation. There are visible tire tracks on the road. A tank is present in the background, partially obscured by dust. No other specified entities like soldiers, checkpoints, vehicles, or guns can be clearly identified in this scene.
/disk3/eric/VG_arma3_roboflow/images/frame_2400.jpg
content : The image depicts a dusty road surrounded by trees and vegetation. There are no visible entities from the ontology provided: soldier, checkpoint, vehicle, gun, or tank.
/disk3/eric/VG_arma3_roboflow/images/frame_2410.jpg
content : The image depicts a dusty road with signs of vehicle tracks. There are trees on both sides, and a sign indicating a curve. None of the specified entities (soldier, checkpoint, vehicle, gun, tank) are visible.
/disk3/eric/VG_arma3_roboflow/images/frame_2420.jpg
content : The image depicts a dusty road surrounded by trees and vegetati

In [11]:
# Get current time
from datetime import datetime, timedelta


current_time = datetime.now()

print("Current Time:", current_time.strftime("%Y-%m-%d %H:%M"))

cnt = 0
all_txts = ""

for k,v in tmp_.items():
    cnt +=1
    current_time += timedelta(minutes=3)
    current_time_v = current_time.strftime("%Y-%m-%d %H:%M")
    print(current_time_v)
    print(v)
    all_txts += current_time_v + ":" 
    all_txts += v

Current Time: 2024-12-23 16:55
2024-12-23 16:58
The image depicts an outdoor scene with a dirt road surrounded by trees and vegetation. There are visible tire tracks on the road. A tank is present in the background, partially obscured by dust. No other specified entities like soldiers, checkpoints, vehicles, or guns can be clearly identified in this scene.
2024-12-23 17:01
The image depicts a dusty road surrounded by trees and vegetation. There are no visible entities from the ontology provided: soldier, checkpoint, vehicle, gun, or tank.
2024-12-23 17:04
The image depicts a dusty road with signs of vehicle tracks. There are trees on both sides, and a sign indicating a curve. None of the specified entities (soldier, checkpoint, vehicle, gun, tank) are visible.
2024-12-23 17:07
The image depicts a dusty road surrounded by trees and vegetation. There is no visible soldier, checkpoint, vehicle, gun, or tank present.
2024-12-23 17:10
The image depicts a dirt road surrounded by trees and ve

In [12]:
all_txts

'2024-12-23 16:58:The image depicts an outdoor scene with a dirt road surrounded by trees and vegetation. There are visible tire tracks on the road. A tank is present in the background, partially obscured by dust. No other specified entities like soldiers, checkpoints, vehicles, or guns can be clearly identified in this scene.2024-12-23 17:01:The image depicts a dusty road surrounded by trees and vegetation. There are no visible entities from the ontology provided: soldier, checkpoint, vehicle, gun, or tank.2024-12-23 17:04:The image depicts a dusty road with signs of vehicle tracks. There are trees on both sides, and a sign indicating a curve. None of the specified entities (soldier, checkpoint, vehicle, gun, tank) are visible.2024-12-23 17:07:The image depicts a dusty road surrounded by trees and vegetation. There is no visible soldier, checkpoint, vehicle, gun, or tank present.2024-12-23 17:10:The image depicts a dirt road surrounded by trees and vegetation. There are no visible ent

In [13]:
!pwd

/home/eric/srcs/Graph_Lab


In [14]:
tmp_["all_txts"] = all_txts

In [15]:
# Save to JSON file
import json
with open('./outputs/all_output_1223_1.json', 'w') as json_file:
    json.dump(tmp_, json_file)

In [16]:
tmp_

defaultdict(None,
            {'frame_2390.jpg': 'The image depicts an outdoor scene with a dirt road surrounded by trees and vegetation. There are visible tire tracks on the road. A tank is present in the background, partially obscured by dust. No other specified entities like soldiers, checkpoints, vehicles, or guns can be clearly identified in this scene.',
             'frame_2400.jpg': 'The image depicts a dusty road surrounded by trees and vegetation. There are no visible entities from the ontology provided: soldier, checkpoint, vehicle, gun, or tank.',
             'frame_2410.jpg': 'The image depicts a dusty road with signs of vehicle tracks. There are trees on both sides, and a sign indicating a curve. None of the specified entities (soldier, checkpoint, vehicle, gun, tank) are visible.',
             'frame_2420.jpg': 'The image depicts a dusty road surrounded by trees and vegetation. There is no visible soldier, checkpoint, vehicle, gun, or tank present.',
             'fram